In [6]:
"""
Downloading VIIRS Active Fire Detections (AFD) with 'earthaccess' python API

For a given geometry (in this case, fire perimeters), download data granules for:

VIIRS AFD Products:
    - VIIRS/NPP Active Fires 6-Min L2 Swath 375m V002 (VNP14IMG)
    - VIIRS/JPSS1 Active Fires 6-Min L2 Swath 375m V002 (VJ1IMG)
VIIRS Geolocation Products:
    - VIIRS/NPP Imagery Resolution Terrain Corrected Geolocation 6-Min L1 Swath 375 m (VNP03IMG)
    - VIIRS/JPSS1 Imagery Resolution Terrain Corrected Geolocation L1 6-Min Swath 375 m (VJ103IMG)

Return: 
    - Downloaded NetCDF granules for the above products
    - GeoDataFrame representing active fire pixel locations and attributes (before geolocation)

Author: maxwell.cook@colorado.edu
"""

import os, time, glob
import earthaccess
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import rasterio as rio
import numpy as np
import math
import gc
import contextlib
import traceback

from netCDF4 import Dataset
from datetime import timedelta
from datetime import datetime

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire'
datadir = os.path.join(maindir,'Aim2/data/spatial/raw/VIIRS/')
dataoutdir = os.path.join(maindir,'Aim2/data/spatial/mod/VIIRS/')

print("Ready !")

Ready !


In [14]:
# Class & Functions !

def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)


class Download_VIIRS_AFD:
    """ Downloads VIIRS Active Fire Data (AFD) for a give GeoPandas GeoDataFrame """
    def __init__(self, start_date, last_date, gdf = gpd.GeoDataFrame(), 
                 geog_crs = 'EPSG:4326', proj_crs = 'EPSG:5070', id_col='fired_id',
                 short_names = ['VNP14IMG', 'VJ114IMG', 'VNP03IMG', 'VJ103IMG'],
                 buffer = None
                ):
        """
        Args:
            - start_date: the intial date for the granule search
            - last_date: the final date for the granule search
            - gdf: GeoDataFrame for search request
            - geog_crs: Geographic projection (to retrieve coordinate pairs in lat/lon)
            - id_col: unique identifier in the GeoDataFrame
            - short_names: the granules to be downloaded
        Returns:
            - Downloaded files (VIIRS Active Fire Data NetCDF and Geolocation information)
            - GeoDataFrame with non-geolocated (raw) fire detections
        """
        
        self.id = gdf[id_col].iloc[0] # grab the unique ID
        self.crs = gdf.crs # the native CRS definition for the input geodataframe
        self.geog_crs = geog_crs
        self.proj_crs = proj_crs
        if buffer is not None:
            self.gdf = gdf
            self.gdf = self.gdf.assign(geometry=self.gdf.buffer(buffer)) # buffer units in meters
        else:
            self.gdf = gdf
        self.bounds = self.gdf.to_crs(geog_crs).unary_union.envelope # for bounds, coords ensure geographic projection
        self.coords = list(self.bounds.exterior.coords)
        self.short_names = short_names
        self.out_dir = os.path.join(datadir, f'FIRED_{self.id}')
        self.date_range = (start_date, last_date)
    
    
    def ea_search_request(self):
        """ generate an earthaccess search request with the given parameters """
        print(f'Fire ID: {self.id}')
        search_dict = {} # to store the search results
        for short_name in self.short_names:
            try:
                # Search for products matching our short names
                result = earthaccess.search_data(
                    short_name=short_name,
                    polygon=self.coords,
                    temporal=self.date_range,
                    count=1000, 
                )
            
                # Check if there is valid data, if not, skip
                if len(result) != 0:
                    # Append the search results data frame to the dictionary
                    search_dict[short_name] = result
                else:
                    raise ValueError(f'No data found for: {short_name} -- Polygon ID {self.id}')
                
            except Exception as e:
                print(f"Skipping polygon ID {self.id}: {short_name}")
                continue

        if not search_dict:
            return None  # Return None for invalid search results
        else:
            return search_dict

    
    def download_results(self, search_dict):
        """ Downloads the search results to directory """
        if search_dict is not None:
            for key, result in search_dict.items():
                # Set the output directory based on short_name
                fd = os.path.join(self.out_dir, f'{key}/')
                if not os.path.exists(fd):
                    os.makedirs(fd)
                if len(os.listdir(fd)) < len(result):
                    # Download the the search results
                    with open(os.devnull, 'w') as f, contextlib.redirect_stdout(f):
                        earthaccess.download(result, local_path=fd)
                else:
                    print("Files already downloaded, skipping ! ")

    
    def create_fire_gdf(self):
        """ Creates a geodataframe with active fire detections from a directory with NetCDF files """

        afd_short_names = ['VNP14IMG', 'VJ114IMG'] # only process the AFD data, not geolocation data
        
        # List of downloaded .nc files
        nc_files = list_files(self.out_dir, "*.nc", recursive=True)
        nc_files = [f for f in nc_files if any(short_name in f for short_name in afd_short_names)]
    
        out_fire_dfs = [] # to store the dataframes for each nc file
        for nc_file in nc_files:
            
            # Read the nc file
            ds = Dataset(nc_file, 'r')

            # Grab some NetCDF attributes
            day_night_flag = ds.getncattr('DayNightFlag')
            short_name = ds.getncattr('ShortName')
            platform = ds.getncattr('PlatformShortName')
            version = ds.getncattr('VersionID')
            start_time_str = ds.getncattr('PGE_StartTime')
            acq_datetime = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f') # convert to datetime
            julian_day = acq_datetime.timetuple().tm_yday # Calculate Julian Day

            # Grab an array of the lat/lons of fire detections
            fire_coords = self.coords
            flats = np.array(ds.variables['FP_latitude'][:])  # lats as np array
            flons = np.array(ds.variables['FP_longitude'][:])  # lons as np array
            fll = np.logical_and.reduce(
                (flons >= fire_coords[0][0], flons <= fire_coords[2][0], flats >= fire_coords[0][1], flats <= fire_coords[2][1]))
    
            # Extract fire pixel information
            lats = flats[fll]
            lons = flons[fll]
            frp = np.array(ds.variables['FP_power'][:])[fll]
            confidence = np.array(ds.variables['FP_confidence'][:])[fll]
            fp_rad13 = np.array(ds.variables['FP_Rad13'][:])[fll]
            fp_t4 = np.array(ds.variables['FP_T4'][:])[fll]
            fp_t5 = np.array(ds.variables['FP_T5'][:])[fll]
            view_az = np.array(ds.variables['FP_ViewAzAng'][:])[fll]
            view_zen = np.array(ds.variables['FP_ViewZenAng'][:])[fll]

            del ds, flats, flons, fll # clean up
    
            # Create a DataFrame with the fire pixel data
            df = pd.DataFrame({
                'fired_id': fire_id,
                'acq_datetime': acq_datetime,
                'acq_julian_day': julian_day,
                'day_night': day_night_flag,
                'short_name': short_name,
                'platform': platform,
                'version': version,
                'latitude': lats,
                'longitude': lons,
                'frp': frp,
                'fp_rad13': fp_rad13,
                'fp_t4': fp_t4,
                'fp_t5': fp_t5,
                'confidence': confidence,
                'view_az_an': view_az,
                'view_zen_an': view_zen
            })
    
            out_fire_dfs.append(df)

            del df # clean up

            gc.collect() # garbage collector
    
        # Concatenate the out dfs
        fire_data = pd.concat(out_fire_dfs) # for the entire fire
        
        # Create a GeoDataFrame
        fp_points = gpd.GeoDataFrame(
            fire_data, 
            geometry=gpd.points_from_xy(fire_data.longitude, fire_data.latitude),
            crs=self.geog_crs) # Geographic coordinates
        # Reproject to projected coordinate system
        fp_points = fp_points.to_crs(self.proj_crs)

        del fire_data

        return fp_points


print("Class and functions ready !")

Class and functions ready !


In [4]:
# Load the fire dataset
fires_path = os.path.join(maindir,'Aim2/data/spatial/mod/FIRED/fired_events_west_aspen.gpkg')
fires = gpd.read_file(fires_path)
print(fires.columns)
print(len(fires))

Index(['fired_id', 'ig_date', 'ig_day', 'ig_month', 'ig_year', 'last_date',
       'event_dur', 'tot_pix', 'tot_ar_km2', 'fsr_px_dy', 'fsr_km2_dy',
       'mx_grw_px', 'mn_grw_px', 'mu_grw_px', 'mx_grw_km2', 'mn_grw_km2',
       'mu_grw_km2', 'mx_grw_dte', 'x', 'y', 'ig_utm_x', 'ig_utm_y', 'lc_code',
       'lc_mode', 'lc_name', 'lc_desc', 'lc_type', 'eco_mode', 'eco_name',
       'eco_type', 'tot_perim', 'pct_aspen', 'geometry'],
      dtype='object')
102


In [15]:
# Get a list of fire IDs
fire_ids = fires['fired_id'].unique()

fp_points = [] # to store the output geodataframes
no_data_ids = [] # to store fire IDs with no data

for fire_id in fire_ids[0:5]:
    fire = fires.loc[fires['fired_id'] == fire_id]
    # Initiate the download and extract class
    downloader = Download_VIIRS_AFD(
        gdf=fire,
        start_date=fire['ig_date'].iloc[0],
        last_date=fire['last_date'].iloc[0],
        buffer=1000, # in meters
    )
    # Retrieve the search results
    try:
        search_results = downloader.ea_search_request()
        if len(search_results) > 0:
            # Downlaod the search results
            downloader.download_results(search_results)
            # Create the active fire detection geodataframe
            fp_points_fire = downloader.create_fire_gdf()
            fp_points.append(fp_points_fire)
            del fp_points_fire
        else:
            raise ValueError(f'No data granules found for {self.id}, skipping completely !')
            
    except Exception as e:
        print(f"Skipping FIRED ID {fire_id}\n{e}")
        traceback.print_exc()  # This will print the full traceback
        no_data_ids.append(fire_id)
        continue  # continue to the next fire id

# Concatenate the results and save out the geodataframe of latlon fire pixels (non-geolocated)
fp_points = gpd.GeoDataFrame(pd.concat(fp_points, ignore_index=True))
fp_points.to_file(outdatadir,'viirs_fp_latlon_points.gpkg')

print("Done!")

Fire ID: 3518
Granules found: 17
Granules found: 16
Granules found: 17
Granules found: 16
Files already downloaded, skipping ! 
Files already downloaded, skipping ! 
Files already downloaded, skipping ! 
Files already downloaded, skipping ! 
Fire ID: 4014
Granules found: 65
Granules found: 61
Granules found: 65
Granules found: 61


QUEUEING TASKS | :   0%|          | 0/65 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/65 [00:00<?, ?it/s]

Files already downloaded, skipping ! 
Files already downloaded, skipping ! 


COLLECTING RESULTS | :   0%|          | 0/65 [00:00<?, ?it/s]

...

COLLECTING RESULTS | :   0%|          | 0/28 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/29 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/29 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/29 [00:00<?, ?it/s]

NameError: name 'outdatadir' is not defined